# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.src.legacy.preprocessing.image import ImageDataGenerator

2024-11-23 07:59:36.005823: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-23 07:59:42.520293: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-23 07:59:47.378146: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732328991.109099    7008 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732328992.120394    7008 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-23 07:59:59.704009: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
tf.__version__
ImageDataGenerator

keras.src.legacy.preprocessing.image.ImageDataGenerator

## Part 1 - Data Preprocessing

### Preprocessing the Training set


This code snippet is part of a process to prepare image data for training a machine learning model using Keras. Here's a breakdown of what each line does:

## Explanation of Code

### 1. **Image Data Augmentation**
```python
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True
)
```
The ImageDataGenerator class in Keras is used to augment image data in real-time and preprocess it. The parameters specified here configure how the data should be augmented.

#### rescale=1./255:

Normalizes the pixel values of the images by scaling them down from the range [0, 255] to [0, 1]. This helps the model train more effectively since normalized data often leads to faster convergence.

#### shear_range=0.2:

Applies random shear transformations to the images. Shearing changes the perspective of an image along a particular axis, making the model more robust to such variations.

#### zoom_range=0.2:

Randomly zooms into the images by up to 20%. This makes the model more robust to scale variations.

#### horizontal_flip=True:

Randomly flips images horizontally. This is useful when horizontal orientation is not a distinguishing factor for the class label (e.g., detecting cats or dogs).

### 2. **training_set = train_datagen.flow_from_directory(...)**

This line generates batches of augmented image data from a directory. Let's break down the parameters:

#### '/home/vpsr/Desktop/python/Machine Learning Resources/dataset/dataset/training_set':

Specifies the directory containing the training images. Inside this directory, images are expected to be organized in subdirectories, where each subdirectory corresponds to a class.

#### target_size=(64, 64):

Resizes all images to 64x64 pixels. This ensures uniformity in input dimensions, which is necessary for training the model.

#### batch_size=32:

Defines the number of images per batch. During training, the model processes images in chunks of 32, rather than one by one, to optimize computational efficiency.

#### class_mode='binary':

Indicates that the classification task is binary (e.g., distinguishing between two classes like "cats" and "dogs"). The labels will be either 0 or 1.

#### What Happens Here?
The ImageDataGenerator applies augmentation and preprocessing to the images in the training set in real-time.
The flow_from_directory method reads images from the specified directory, resizes them, applies the defined augmentations, and generates batches of data for training.

#### Output
This setup returns a Python generator (training_set) that can be used directly with the fit or fit_generator method of a Keras model. The generator ensures efficient memory usage by loading and processing only a batch of images at a time instead of loading the entire dataset into memory.

#### Purpose of Data Augmentation
The goal of augmentation is to artificially increase the diversity of the training dataset by introducing transformations. This helps prevent overfitting, especially when the dataset is small.


In [3]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True)
training_set = train_datagen.flow_from_directory('/home/vpsr/Desktop/python/Machine Learning Resources/dataset/dataset/training_set', 
                                                 target_size=(64,64), 
                                                 batch_size=32, 
                                                 class_mode='binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

The `ImageDataGenerator` class is used here to preprocess the test images. The only parameter specified is:

#### rescale=1./255:
Normalizes pixel values from [0, 255] to [0, 1]. This is crucial for maintaining consistency with the preprocessing applied to the training data.

**Note:** Unlike the training data generator, no data augmentation (like shearing or flipping) is applied here. The test set should represent real-world data without modifications, ensuring accurate evaluation.

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory('/home/vpsr/Desktop/python/Machine Learning Resources/dataset/dataset/test_set', 
                                            target_size=(64,64), 
                                            batch_size=32, 
                                            class_mode='binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64,64,3]))

/home/vpsr/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-11-23 08:00:58.326372: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


### Step 2 - Pooling

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [11]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [13]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [14]:
cnn.fit(x=training_set, validation_data=test_set, epochs=25)

/home/vpsr/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25


2024-11-23 10:59:20.663344: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 15745024 exceeds 10% of free system memory.


  2/250 ━━━━━━━━━━━━━━━━━━━━ 24s 98ms/step - accuracy: 0.5234 - loss: 0.7741

2024-11-23 10:59:22.549347: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 28096128 exceeds 10% of free system memory.
2024-11-23 10:59:22.561870: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 28096128 exceeds 10% of free system memory.
2024-11-23 10:59:22.581012: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 15745024 exceeds 10% of free system memory.
2024-11-23 10:59:22.597419: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 14530320 exceeds 10% of free system memory.


250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 542ms/step - accuracy: 0.5446 - loss: 0.6934

/home/vpsr/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 163s 631ms/step - accuracy: 0.5448 - loss: 0.6933 - val_accuracy: 0.6555 - val_loss: 0.6295
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 41s 165ms/step - accuracy: 0.6567 - loss: 0.6292 - val_accuracy: 0.6660 - val_loss: 0.6234
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 42s 167ms/step - accuracy: 0.6982 - loss: 0.5819 - val_accuracy: 0.7120 - val_loss: 0.5641
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 42s 167ms/step - accuracy: 0.7175 - loss: 0.5456 - val_accuracy: 0.7250 - val_loss: 0.5508
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 42s 169ms/step - accuracy: 0.7303 - loss: 0.5253 - val_accuracy: 0.7305 - val_loss: 0.5472
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 42s 169ms/step - accuracy: 0.7569 - loss: 0.5065 - val_accuracy: 0.7670 - val_loss: 0.4923
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 44s 176ms/step - accuracy: 0.7628 - loss: 0.4865 - val_accuracy: 0.7675 - val_loss: 0.4968
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 44s 176ms/step - accuracy: 0.7813 - loss: 0.4659 - va

## Part 4 - Making a single prediction

In [17]:
import numpy as np
from tensorflow.keras.preprocessing import image
# from keras.src.legacy.preprocessing import image

In [77]:
test_image = image.load_img('/home/vpsr/Desktop/python/Machine Learning Resources/dataset/dataset/test_set/cats/cat.4018.jpg',target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = cnn.predict(test_image)
# print(training_set.class_indices)
if result[0][0]==1:
    prediction='dog'
else:
    prediction='cat'
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
cat


In [78]:
print(result)

[[0.]]
